<a href="https://colab.research.google.com/github/PigStep/furniture-store-extraction/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

#For parsing
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
urls_df = pd.read_csv("URL_list.csv")
urls_df.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [3]:
url_series = pd.Series(urls_df["max(page)"]) #To make code easier
url_series.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


# Предобработка данных
Сохраним только открывающиеся ссылки

In [4]:
def check_is_url_parsing(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  try:
      response = requests.get(url, headers=headers, timeout=10)
      response.raise_for_status() #check for html error
      soup = BeautifulSoup(response.text, "html.parser")

      return True

  except requests.exceptions.RequestException as e:
    print(f"Attemp to URL falied: {e}")
    return False

In [5]:
def create_good_urls_csv():
  """Создает файл 'ParsingURL_list.csv' с посещаемыми ссылками"""
  good_urls =[]
  for url in url_series:
    if(check_is_url_parsing(url)):
      good_urls.append(url)

  urls = pd.Series(good_urls, name="url")
  urls.to_csv("ParsingURL_list.csv")
  print("Dataset have been created and saved")

In [6]:
# create_good_urls_csv()

In [68]:
# Пересоздадим серию c очищенными ссылками
url_series = pd.read_csv("ParsingURL_list.csv")
url_series = pd.Series(url_series["url"])

#Parsing data from sites

# Создадим "идеальную выборку"
Некоторые сайты имеют application json теги для системы поисковиков. Их очень удобно парсить и они почти не имеют шума

In [8]:
def get_parsed_data(url):
    """Получает данные о продукте из JSON-LD разметки страницы"""
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        scripts = soup.find_all("script", type="application/ld+json")
        product = None

        for script in scripts:
            try:
                data = json.loads(script.string)

                if isinstance(data, list):
                    for obj in data:
                        if isinstance(obj, dict) and obj.get("@type") == "Product":
                            product = obj
                            break
                elif isinstance(data, dict) and data.get("@type") == "Product":
                    product = data

                if product:
                    print(product)

                    offers = product.get("offers", {})
                    if isinstance(offers, list):
                        offers = offers[0] if offers else {}
                    price = offers.get("price")

                    # Чтобы предотвратить ошибки типов
                    brand = product.get("brand")
                    brand_name = brand if isinstance(brand, str) else (brand.get("name") if isinstance(brand, dict) else None)

                    return {
                        "url": url,
                        "title": product.get("name"),
                        "description": product.get("description"),
                        "price": price,
                        "brand": brand_name
                    }

            except json.JSONDecodeError:
                continue

        print(f'No product data found in JSON-LD: <a href="{url}">{quote(url)}</a>')
        return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [9]:
def get_cleaned_data():
  all_data =[]
  for url in url_series:
    product_data = get_parsed_data(url)

    if(product_data):
      product_data["url"] = url
      all_data.append(product_data)

  # Создаем датафрейм разом
  urls_df = pd.DataFrame(all_data)
  urls_df.to_csv("JSON_train_data.csv")
  print("Dataset created and saved succseffully as 'JSON_train_data.csv'")

# get_cleaned_data()


In [12]:
#Создадим датасет для ручной разметки
manual_df = pd.read_csv("JSON_train_data.csv",index_col=0)
manual_df.head()

,url,title,description,price,brand
0,https://www.factorybuys.com.au/products/euro-t...,Factory Buys 32cm Euro Top Mattress - King,Pamper yourself with Factory Buys Euro Top Mat...,466.2,Factory Buys
1,https://dunlin.com.au/products/beadlight-cirrus,Beadlight Cirrus LED Reading Light,"Handmade by Beadlight in the UK, the Cirrus co...",1758.0,Beadlight
2,https://themodern.net.au/products/hamar-plant-...,Hamar Plant Stand - Ash,The Hamar Plant Stand is a wonderful accessory...,412.0,The Modern
3,https://vastinterior.com.au/products/samson-da...,Samson Daybed Single,\nNote: This is a frame only. Does not come wi...,899.0,Samson
4,https://www.hudsonfurniture.com.au/products/st...,String weave timber stool,Our Hudson String weave stool is made from nat...,495.0,Hudson Furniture


In [32]:
manual_df["price"] = "$"+manual_df.price

In [118]:
with open("annote.txt", "w", encoding="utf-8") as file:
    for i in range(0,51):
      for text in manual_df.drop("url",axis=1).iloc[i]:
        file.write(str(text)+" ")
      file.write("\n")


In [55]:
url = quote(manual_df.iloc[0].url)
print(f'<a href="{url}">{url}</a>')

<a href="https%3A//claytongrayhome.com/products/palecek-coco-ruffle-mirror">https%3A//claytongrayhome.com/products/palecek-coco-ruffle-mirror</a>


# Создадим "подшумленную выборку"
В дополнение к "идеальным" данным стоит добавить немного зашумленные, чтобы улучшить общее качество обучения

In [62]:
pip install requests beautifulsoup4 lxml trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.8 MB/s eta 0:00:00


In [115]:
import requests
import trafilatura
from bs4 import BeautifulSoup
import re

def extract_main_text(html):
    return trafilatura.extract(html, include_tables=True, include_links=False)

def extract_price_from_html(html):
    soup = BeautifulSoup(html, "lxml")

    # Удалим мусорные теги
    for tag in soup(["script", "style", "footer", "nav", "aside", "form", "svg"]):
        tag.decompose()

    # Шаблоны цен
    price_patterns = [
        r"(?<!\w)(?:\$|USD|€|EUR|₽|руб\.?|RUB)\s?\d{1,5}(?:[.,]\d{2})?(?!\w)",
        r"\d{1,5}(?:[.,]\d{2})?\s?(?:руб\.?|₽|RUB|USD|$|€|EUR)"
    ]

    # 1. Поиск по тексту
    text = soup.get_text(separator=" ")
    for pattern in price_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(0).strip()

    # 2. Поиск по классам
    price_tags = soup.find_all(class_=re.compile(r"price|amount|value|cost", re.I))
    for tag in price_tags:
        price_text = tag.get_text(strip=True)
        for pattern in price_patterns:
            match = re.search(pattern, price_text, re.IGNORECASE)
            if match:
                return match.group(0).strip()

    return ""

def extract_title(html):
    soup = BeautifulSoup(html, "lxml")
    if soup.find(["h1","h2","h3"]):
        return soup.find(["h1","h2","h3"]).get_text(strip=True)
    elif soup.title:
        return soup.title.get_text(strip=True)
    return ""

def extract_full_product_text(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    html = response.text

    name = extract_title(html)
    price = extract_price_from_html(html)
    body = extract_main_text(html)

    parts = []
    if name:
        parts.append(name)
    if price:
        parts.append(price)
    if body:
        parts.append(body)

    return '\n\n'.join(parts)


In [119]:
url = url_series[45]
data = extract_full_product_text(url)
print(json.dumps(data, indent=2, ensure_ascii=False))

"Just added to your cart\n\nElegantly sculptural, the Molloy collection of dining tables, chairs, shelving and coffee tables feature fluid detailing inspired by a rare act of nature.\nThe idea for the artfully rounded joinery, where two elements meet and seamlessly become one, comes from the unusual meeting of two rivers to form the Molloy River in Western Australia – a place where Adam Goodrum hand-built a holiday home with his family. The Molloy collection is available in a variety of timber finishes.\nWinner of the Good Design Awards 2016.\n52W x 50D x 78H cm\n45cm seat height\nDownload NAU Standards Finishes and Upholstery Overview\nSolid timber in oak, walnut, ash or black stained ash.\nA firm believer that every environment is defined by the objects within it, Adam Goodrum designs with the philosophy that an object must therefore justify its existence – through its story and detailing. For this reason, his designs celebrate process and craftsmanship, and accentuate components and